In [12]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.optimizers import Adam

In [2]:
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
batch_size = 32
epochs = 10

In [3]:
train_data_dir = 'train/' 

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

In [6]:
for layer in base_model.layers:
    layer.trainable = False

In [5]:
model = Sequential([
    base_model,
    
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [7]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
with tf.device('/GPU:0'):
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        epochs=epochs
    )

Epoch 1/10
625/625 [==============================] - 653s 1s/step - loss: 0.3003 - accuracy: 0.8676 - val_loss: 0.1971 - val_accuracy: 0.9159
Epoch 2/10
625/625 [==============================] - 496s 793ms/step - loss: 0.1882 - accuracy: 0.9224 - val_loss: 0.1769 - val_accuracy: 0.9289
Epoch 3/10
625/625 [==============================] - 529s 847ms/step - loss: 0.1689 - accuracy: 0.9307 - val_loss: 0.1826 - val_accuracy: 0.9265
Epoch 4/10
625/625 [==============================] - 524s 838ms/step - loss: 0.1583 - accuracy: 0.9367 - val_loss: 0.1686 - val_accuracy: 0.9275
Epoch 5/10
625/625 [==============================] - 542s 867ms/step - loss: 0.1398 - accuracy: 0.9428 - val_loss: 0.1578 - val_accuracy: 0.9399
Epoch 6/10
625/625 [==============================] - 549s 878ms/step - loss: 0.1382 - accuracy: 0.9423 - val_loss: 0.1611 - val_accuracy: 0.9347
Epoch 7/10
625/625 [==============================] - 585s 936ms/step - loss: 0.1293 - accuracy: 0.9475 - val_loss: 0.1899 - va

In [22]:
model.save('dog_vs_cat_cnn_vgg16.h5')

In [26]:
img_path = 'cat.jpg'  
img = load_img(img_path, target_size=(224, 224))  # Resize the image to the target size
img_array = img_to_array(img)  # Convert the image to a numpy array
img_array = tf.expand_dims(img_array, axis=0)  # Add a batch dimension
img_array = img_array / 255.0  # Rescale the image if you used rescaling during training

In [27]:
prediction = model.predict(img_array)

1/1 [==============================] - 0s 50ms/step


In [28]:
if prediction[0] > 0.5:
    print("It's a dog!")
else:
    print("It's a cat!")

It's a cat!
